# Instalación del ambiente

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Descarga y unzip de dataset

In [ ]:
# !kaggle competitions download -c aerial-cactus-identification
# import os
# os.chdir('/content/competitions/aerial-cactus-identification')
# !unzip -q test.zip
# !unzip -q train.zip

import os
# os.chdir('../input/Aerial Cactus Identification')
# print(os.listdir("../input/Aerial Cactus Identification"))
# print(os.listdir("../input/train"))
# os.chdir('.')

# !unzip -q test.zip
# !unzip -q train.zip

# Comienza implementación del Kernel

In [ ]:
import pandas as pd
train_ds = pd.read_csv('../input/train.csv', dtype=str)
train_ds.head()

In [ ]:
# se inspecciona que hay en los directorios
train_dir = os.path.join('../input/train/train')
print('total training images:', len(os.listdir(train_dir)))

train_files = os.listdir(train_dir)
print(train_files[:10])


In [ ]:
#se muestran un par de imagenes
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_img = [os.path.join(train_dir, fname) 
                for fname in train_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_img):
  print(img_path)
  img = mpimg.imread(img_path)
  print(img.shape)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator


# codigo copiado del clasificador de piedra papel y tijera de la io 
# https://colab.research.google.com/github/lmoroney/io19/blob/master/Zero%20to%20Hero/Rock-Paper-Scissors.ipynb#scrollTo=LWTisYLQM1aM

TRAINING_DIR = "../input/train/train/"
training_datagen = ImageDataGenerator(
  rescale = 1./255,
  rotation_range=40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest',
  validation_split=0.25
)


train_generator = training_datagen.flow_from_dataframe(
  dataframe=train_ds, 
  directory=TRAINING_DIR, 
  shuffle=True,
  x_col="id", 
  y_col="has_cactus", 
  target_size=(32,32), 
  class_mode="categorical",
  subset="training"
)

validation_generator = training_datagen.flow_from_dataframe(
  dataframe=train_ds, 
  directory=TRAINING_DIR, 
  shuffle=True,
  x_col="id", 
  y_col="has_cactus", 
  target_size=(32,32), 
  class_mode="categorical",
  subset="validation"
)

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    
#     tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(32, 32, 3)),
    #tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# logdir = 'logs'
# tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

history = model.fit_generator(train_generator, epochs=32, validation_data = validation_generator, verbose = 1) #, callbacks=[tensorboard_callback])

# model.save("rps.h5")
  

In [ ]:
# # Load TENSORBOARD
# %load_ext tensorboard.notebook
# # Start TENSORBOARD
# %tensorboard --logdir logs

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation Loss')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
import numpy as np 

testdf = pd.read_csv("../input/sample_submission.csv", dtype=str)
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_dataframe(
  dataframe=testdf,
  directory='../input/test/test/',
  target_size=(32,32),
  x_col="id",
  y_col=None,
  class_mode=None,
#   class_mode="categorical",
  shuffle=False,
)
test_generator.reset()
pred = model.predict_generator(test_generator, verbose=1)


predicted_class_indices = np.argmax(pred,axis=1)

# print(predicted_class_indices)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames
results=pd.DataFrame({"id":filenames,
                      "has_cactus":predictions})
results.to_csv("submission.csv",index=False)